## Requesting Instrument Asset Information

In this brief demo you will learn:

* how to request asset information for an instrument, such as deployment start and end dates, lat, lon and depth

Below we have entered credential for a default user. These should work, unless they have been reset.  
Alternatively, you can login in at https://ooinet.oceanobservatories.org/ and obtain a API username and API token under your profile (top right corner).

In [1]:
username = 'OOIAPI-D8S960UXPK4K03'
token = 'IXL48EQ2XY'

### Pulling asset information into your notebook.

The ingredients being used to build the asset_info_url can be found here. For this example, we will pull in the asset information for the CTD at the Axial Shallow Profiler http://ooi.visualocean.net/instruments/view/RS03AXPS-PC03A-4A-CTDPFA303

In [3]:
import requests
import datetime
import pandas as pd

In [4]:
subsite = 'RS03AXPS'
node = 'PC03A'
sensor = '4A-CTDPFA303'

Now let's build the asset_info_url and request the information

In [5]:
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12587/events/deployment/inv/'

asset_info_url ='/'.join((base_url, subsite, node, sensor, '-1'))

r = requests.get(asset_info_url,auth=(username, token))
asset_info = r.json()

In [6]:
asset_info[0]['location']

{'depth': 190.0,
 'location': [-129.75341, 45.8305],
 'latitude': 45.8305,
 'longitude': -129.75341,
 'orbitRadius': None}

Similar to an synchronous json data request, the asset information is returned as a list of json object. In the following example we will pull out the sart and end time for each deployment of this asset.

In [8]:
ref_des_list = []
start_time_list = []
end_time_list = []
deployment_list = []
depth_list = []
lat_list = [] 
lon_list = [] 

for i in range(len(asset_info)):
    refdes = asset_info[i]['referenceDesignator']
    ref_des_list.append(refdes)

    deployment = asset_info[i]['deploymentNumber']
    deployment_list.append(deployment)

    start = asset_info[i]['eventStartTime']
    end = asset_info[i]['eventStopTime']

    try:
        start_time = datetime.datetime.utcfromtimestamp(start/1000.0)
        start_time_list.append(start_time)

        end_time = datetime.datetime.utcfromtimestamp(end/1000.0)
        end_time_list.append(end_time)

    except:
        end_time = datetime.datetime.utcnow()
        end_time_list.append(end_time)
        
    depth = asset_info[i]['location']['depth']
    depth_list.append(depth)
    lat = asset_info[i]['location']['latitude']
    lat_list.append(lat)
    lon = asset_info[i]['location']['longitude']
    lon_list.append(lon)
    

data_dict = {
    'refdes':ref_des_list,
    'deployment':deployment_list,
    'start_time':start_time_list,
    'end_time':end_time_list,
    'depth': depth_list,
    'latitutde':lat_list,
    'longitude':lon_list}

deployment_data = pd.DataFrame(data_dict, columns = ['refdes', 'deployment','start_time', 'end_time'])

deployment_data

,refdes,deployment,start_time,end_time
0,RS03AXPS-PC03A-4A-CTDPFA303,1,2014-09-27 13:29:00,2015-07-09 00:00:00.000000
1,RS03AXPS-PC03A-4A-CTDPFA303,2,2015-07-09 00:07:00,2016-07-14 00:00:00.000000
2,RS03AXPS-PC03A-4A-CTDPFA303,3,2016-07-14 21:18:00,2017-07-30 14:00:00.000000
3,RS03AXPS-PC03A-4A-CTDPFA303,4,2017-07-31 04:00:00,2018-04-25 23:17:41.610199
